In [1]:
from ttig.models.vqgan_clip import VQGANConfig, VqGanClipGenerator

In [2]:
config = VQGANConfig()
im_gen = VqGanClipGenerator('checkpoints/vqgan/vqgan_imagenet_f16_16384.ckpt', 'checkpoints/vqgan/vqgan_imagenet_f16_16384.yaml', config, 'ViT-B/16')

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.
Restored from checkpoints/vqgan/vqgan_imagenet_f16_16384.ckpt


In [3]:
import torch
from torch.nn.functional import one_hot

texts = ["a child's drawing of the universal library", "the universal library trending on artstation"]
batch_size = len(texts)
f = 2 ** (im_gen.vqgan.decoder.num_resolutions - 1) # TODO: What number is this usually?
toksX, toksY = im_gen.config.size[0] // f, im_gen.config.size[1] // f
#sides = (toksX * f, toksY * f)
# TODO: What is this?????
e_dim = im_gen.vqgan.quantize.e_dim
n_toks = im_gen.vqgan.quantize.n_e
z_min = im_gen.vqgan.quantize.embedding.weight.min(dim=0).values[None, :, None, None]
z_max = im_gen.vqgan.quantize.embedding.weight.max(dim=0).values[None, :, None, None]

one_hot_embeds = one_hot(torch.randint(n_toks, [batch_size, toksY * toksX], device='cuda:0'), n_toks).float()
z = one_hot_embeds @ im_gen.vqgan.quantize.embedding.weight
z = z.view([-1, toksY, toksX, e_dim]).permute(0, 3, 1, 2) 

In [4]:
print(im_gen.l2_norm(z, 0))

tensor([0.0179, 0.0298], device='cuda:0')


In [5]:
texts = [
    "a child's drawing of the universal library",
    "an ink drawing of the universal library",
    "a photorealistic image of the universal library",
    "the universal library trending on artstation"]
im_gen.config.num_cuts = 16
images = im_gen.generate(texts)

  0%|                                                                                                             | 0/500 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (1024) must match the size of tensor b (512) at non-singleton dimension 2

In [ ]:
from torchvision.transforms.functional import to_pil_image

im0 = to_pil_image(images[0])
im2 = to_pil_image(images[1])
im0.save('im0.png')